# Urban Sound 8K

In [4]:
import sys
sys.path.append('../..')
from lib_data_preprocess import *
%matplotlib inline

DATAROOT = Path('/mnt/dataset/UrbanSound8K')
df = pd.read_csv(DATAROOT/'metadata/UrbanSound8K.csv')
df_train = df[~df.fold.isin([4, 5])]
df_test  = df[df.fold.isin([4, 5])]

train_files = [str(DATAROOT/'audio'/('fold%d' % (r.fold))/r.slice_file_name) for i, r in df_train.iterrows()]
plain_y_train = [conf.label2int[r['class']] for i, r in df_train.iterrows()]
test_files  = [str(DATAROOT/'audio'/('fold%d' % (r.fold))/r.slice_file_name) for i, r in df_test.iterrows()]
plain_y_test = [conf.label2int[r['class']] for i, r in df_test.iterrows()]

In [5]:
XX = mels_build_multiplexed_X(conf, train_files)
X_train, y_train = mels_demux_XX_y(XX, plain_y_train)
XX = mels_build_multiplexed_X(conf, test_files)
X_test, y_test = mels_demux_XX_y(XX, plain_y_test)

In [6]:
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)
[a.shape for a in [X_train, y_train, X_test, y_test]]

[(34030, 128, 128, 1), (34030,), (9630, 128, 128, 1), (9630,)]

## Inside this dataset

In [7]:
df_train[:10]

,slice_file_name,fsID,start,end,salience,fold,classID,class
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn
10,100648-1-1-0.wav,100648,8.998279,10.052132,2,10,1,car_horn
11,100648-1-2-0.wav,100648,16.699509,17.104837,2,10,1,car_horn
12,100648-1-3-0.wav,100648,17.631764,19.253075,2,10,1,car_horn
13,100648-1-4-0.wav,100648,25.332994,27.197502,2,10,1,car_horn
14,100652-3-0-0.wav,100652,0.000000,4.000000,1,2,3,dog_bark
15,100652-3-0-1.wav,100652,0.500000,4.500000,1,2,3,dog_bark
16,100652-3-0-2.wav,100652,1.000000,5.000000,1,2,3,dog_bark
17,100652-3-0-3.wav,100652,1.500000,5.500000,1,2,3,dog_bark
18,100795-3-0-0.wav,100795,0.191790,4.191790,1,10,3,dog_bark


In [20]:
(df.end - df.start).describe()

count    8732.000000
mean        3.607904
std         0.973570
min         0.054517
25%         4.000000
50%         4.000000
75%         4.000000
max         4.000000
dtype: float64

## Check duplication with FSDKaggle2018

In [9]:
fsd = pd.read_csv('~/.kaggle/competitions/freesound-audio-tagging/train_post_competition.csv')

In [17]:
usids = np.array(df.fsID.unique(), dtype=np.int)
fsids = np.array(fsd.freesound_id.unique(), dtype=np.int)
dup_ids = [uid for uid in usids if uid in fsids]
df[df.fsID.isin(dup_ids)].fold.unique()

array([ 5,  2, 10,  1,  4,  3,  8,  6,  9,  7])